In [ ]:
!pip install pypdf2
!pip install transformers
!pip install torch

In [ ]:
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
from string import punctuation
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
reader = PdfReader('W5 ETK Notes.pdf')
text = []
for page in reader.pages:
  text.append(nltk.sent_tokenize(page.extract_text()))

In [ ]:
split_text=[]
for i in text:
  for j in i:
    split_text.append(j)

In [ ]:
def cleanup(text):
  ltext=text.lower()
  ltext = re.sub(f"[{re.escape(punctuation)}]", "", ltext)
  ltext = " ".join(ltext.split())
  remove_chars=['’', ',','‘','•','”','“','—']
  for i in remove_chars:
    ltext=ltext.replace(i, '')
  clean_text = " ".join(ltext.split())

  cl_text=clean_text.split()
  la=""
  for i in cl_text:
    la+=i+" "
  laa=""
  laa+="".join([char for char in la if not unicodedata.category(char) == 'Co'])
  return laa
chunk_list=[]
generated_summaries = []
for i in split_text:
  cleaned_text=cleanup(i)
  chunk_list.append(cleaned_text)

In [ ]:
length=0
nested_list=[]
sent_list=[]
generated_summaries = []
for i in chunk_list:
  length+=len(i)
  if length<1024:
    sent_list.append(i)
  else:
    nested_list.append(sent_list)
    sent_list=[i]
    length=0
if sent_list:
  nested_list.append(sent_list)

In [ ]:
torch_device = 'cuda'
generated_summaries = []

for i in nested_list:
    inputs = tokenizer(' '.join(i), max_length=1000, return_tensors='pt', truncation=True).to(torch_device)
    inputs = {key: val.to(torch_device) for key, val in inputs.items()}
    summary_ids = model.to(torch_device).generate(inputs['input_ids'])

    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    generated_summaries.append(summary)


In [ ]:
generated_summaries

[['hello learners in this session we are going to discuss on the indian music and its divisions. The three art forms music dance and drama have been an integral aspects of indian culture art reflects human emotions and human beings spontaneously express their frame of mind through various art forms  singing dancing acting are attributes of performing arts.'],
 ['The earliest text dealing exclusively with performing arts is bharata s natyashashtra compiled between second century bc and second century ad. Many rulers such as samudragupta king bhoja of dhara and king someshavra of kalyana patronised music. Music was also associated with the worship of gods and goddess in the temples.'],
 ['The musicians of hindustani classical music are usually associated to a gharana or a particular style of music. gharanas refer to hereditary linkages of musicians which represent the core of the style and distinguish them from the other. The most famous are sitar sarod santoor, sarangi, pakhawaj tabla a

In [ ]:
summarizedText=""
for i in generated_summaries:
  for j in i:
    summarizedText+=j+" "

In [ ]:
summarizedText

"hello learners in this session we are going to discuss on the indian music and its divisions. The three art forms music dance and drama have been an integral aspects of indian culture art reflects human emotions and human beings spontaneously express their frame of mind through various art forms  singing dancing acting are attributes of performing arts. The earliest text dealing exclusively with performing arts is bharata s natyashashtra compiled between second century bc and second century ad. Many rulers such as samudragupta king bhoja of dhara and king someshavra of kalyana patronised music. Music was also associated with the worship of gods and goddess in the temples. The musicians of hindustani classical music are usually associated to a gharana or a particular style of music. gharanas refer to hereditary linkages of musicians which represent the core of the style and distinguish them from the other. The most famous are sitar sarod santoor, sarangi, pakhawaj tabla and mridangam. 